In [10]:
pip install ipython-sql


   ---------------------------------------- 0.0/44.0 kB ? eta -:--:--
   ---------------------------------------- 44.0/44.0 kB ? eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
Note: you may need to restart the kernel to use updated packages.


In [11]:
%load_ext sql


In [12]:
%sql sqlite:///school.db


In [13]:
%%sql

CREATE TABLE IF NOT EXISTS Students (
    student_id INTEGER PRIMARY KEY,
    first_name TEXT NOT NULL,
    last_name TEXT NOT NULL,
    birthdate DATE
);

CREATE TABLE IF NOT EXISTS Courses (
    course_id INTEGER PRIMARY KEY,
    course_name TEXT NOT NULL,
    instructor_name TEXT NOT NULL
);

CREATE TABLE IF NOT EXISTS Enrollments (
    enrollment_id INTEGER PRIMARY KEY,
    student_id INTEGER,
    course_id INTEGER,
    enrollment_date DATE,
    FOREIGN KEY (student_id) REFERENCES Students (student_id),
    FOREIGN KEY (course_id) REFERENCES Courses (course_id)
);


 * sqlite:///school.db
Done.
Done.
Done.


[]

In [14]:
%%sql

INSERT INTO Students (student_id, first_name, last_name, birthdate) VALUES
(1, 'John', 'Doe', '2000-01-15'),
(2, 'Jane', 'Smith', '1999-05-23'),
(3, 'Mike', 'Brown', '2001-07-30')
ON CONFLICT(student_id) DO NOTHING;

INSERT INTO Courses (course_id, course_name, instructor_name) VALUES
(1, 'Mathematics', 'Dr. Alice'),
(2, 'Physics', 'Dr. Bob'),
(3, 'Chemistry', 'Dr. Carol')
ON CONFLICT(course_id) DO NOTHING;

INSERT INTO Enrollments (enrollment_id, student_id, course_id, enrollment_date) VALUES
(1, 1, 1, '2023-09-01'),
(2, 1, 2, '2023-09-01'),
(3, 2, 1, '2023-09-02'),
(4, 3, 3, '2023-09-03')
ON CONFLICT(enrollment_id) DO NOTHING;


 * sqlite:///school.db
0 rows affected.
0 rows affected.
0 rows affected.


[]

In [ ]:
%sql SELECT s.first_name, s.last_name FROM Students s JOIN Enrollments e ON
s.student_id = e.student_id JOIN Courses c ON e.course_id = c.course_id WHERE c.course_name = 'Mathematics';


Get the names of all students enrolled in the Mathematics course:


joining three tables

In [21]:
%%sql SELECT first_name,last_name from Students s JOIN Enrollments e on s.student_id = e.student_id JOIN Courses c on c.course_id = e.course_id 
WHERE course_name = 'Mathematics'

 * sqlite:///school.db
Done.


first_name,last_name
John,Doe
Jane,Smith


Count the number of students enrolled in each course:

In [29]:
%sql SELECT c.course_name,count(s.student_id) AS 'num_students' from Courses c JOIN Enrollments e on c.course_id=e.course_id JOIN Students s on e.student_id=s.student_id GROUP BY c.course_name order by num_students

 * sqlite:///school.db
Done.


course_name,num_students
Chemistry,1
Physics,1
Mathematics,2


List all students along with their enrolled courses:




In [33]:
%sql SELECT s.first_name,s.last_name,c.course_name from Courses c JOIN Enrollments e on c.course_id=e.course_id JOIN Students s on e.student_id=s.student_id 

 * sqlite:///school.db
Done.


first_name,last_name,course_name
John,Doe,Mathematics
John,Doe,Physics
Jane,Smith,Mathematics
Mike,Brown,Chemistry


List all courses that do not have any students enrolled:



In [34]:
%sql SELECT c.course_name  from Courses c JOIN Enrollments e on c.course_id=e.course_id JOIN Students s on e.student_id=s.student_id where e.enrollment_id = NULL

 * sqlite:///school.db
Done.


course_name


Find the instructor who teaches the most courses:



In [42]:
%sql SELECT instructor_name from Courses where course_id = (SELECT course_id from enrollments Group by course_id order by count(student_ID) DESC LIMIT 1) 

 * sqlite:///school.db
Done.


instructor_name
Dr. Alice


List all students who are enrolled in more than one course:



In [49]:
%sql SELECT s.first_name,s.last_name,COUNT(e.course_id) AS 'num_courses' from Students s JOIN Enrollments e on s.student_id=e.student_id JOIN Courses c on c.course_id=e.course_id GROUP BY s.student_id HAVING num_courses>1;

 * sqlite:///school.db
Done.


first_name,last_name,num_courses
John,Doe,2


Find the students who are not enrolled in any course:

In [55]:
%sql SELECT s.first_name,s.last_name from Students s JOIN Enrollments e on s.student_id=e.student_id JOIN Courses c on e.course_id=c.course_id WHERE e.enrollment_id = NULL

 * sqlite:///school.db
Done.


first_name,last_name
